<h1>The following notebook contains the scripts necessary for creating the endpoints and deploying the model to Azure ML</h2>
<p> All of the cells must be executed for the model to be successfully deployed </p>

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

<h5>The following cell contains senstive environment variables and auth details and are therefore labeled as such. Please fill them out 
accordingly before executing the cell </h5>

In [2]:
# enter details of your AzureML workspace
subscription_id = "cdc162f9-b8e0-4d8f-9712-62348a4d0c5f"
resource_group = "boney0310-rg"
workspace = "Maatran"

In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

<h1>ONLINE DEPLOYMENT</h1>

In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Endpoint for accessing or consuming the Maatran pregnancy risk classifier",
    auth_mode="key",
    tags={"v": 1.0},
)

In [5]:
#Deploy the online endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint)

In [6]:
#Register the model
model = Model(path="../model/pregnancy_risk_classifier.pkl")

#Register the environment and the Docker img on which the model will run
env = Environment(
    conda_file="../model/environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

#Create the deployment and register it with the previously created endpoint, model and environment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../model/onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [7]:
#Deploy the model
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-02070203078896 exists
Uploading onlinescoring (0.0 MBs): 100%|##########| 1276/1276 [00:00<00:00, 11783.70it/s]


Uploading pregnancy_risk_classifier.pkl (< 1 MB): 100%|##########| 5.00M/5.00M [00:07<00:00, 669kB/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..............................................................

In [8]:
#Check status of the endpoint
ml_client.online_endpoints.get(name=online_endpoint_name)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-02070203078896.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-02070203078896.centralindia.inference.ml.azure.com/swagger.json', 'name': 'endpoint-02070203078896', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/cdc162f9-b8e0-4d8f-9712-62348a4d0c5f/resourcegroups/boney0310-rg/providers/microsoft.machinelearningservices/workspaces/maatran/onlineendpoints/endpoint-02070203078896', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/cdc162f9-b8e0-4d8f-9712-62348a4d0c5f/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:e84d0296-cfb8-4fc7-9550-f200ef2e4919:b0762ff9-0c27-48df-9262-eb9229eeb830?api-version=2022-02-01-preview'}, 'id': '/subscriptions/cdc162f9-b8e0-4d8f-9712-62348a4d0c5f/resourceGrou

In [13]:
#Print all deployed endpoints
print("Kind\tLocation\tName")
print("-------\t----------\t------------------------")
for endpoint in ml_client.online_endpoints.list():
    print(f"{endpoint.kind}\t{endpoint.location}\t{endpoint.name}")

Kind	Location	Name
-------	----------	------------------------
Managed	centralindia	endpoint-02070203078896


In [10]:
#Get logs of the deployment
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Pulling, Type: Normal, Time: 2023-02-06T20:40:48.18905Z, Message: Start pulling container image\nKind: Pod, Name: Downloading, Type: Normal, Time: 2023-02-06T20:40:48.745281Z, Message: Start downloading models\nKind: Pod, Name: Pulled, Type: Normal, Time: 2023-02-06T20:41:41.44641Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2023-02-06T20:41:41.44641Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2023-02-06T20:41:41.672125Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2023-02-06T20:41:41.784746Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2023-02-06T20:42:00.654256852Z, Message: Container is ready\n\nContainer logs:\

In [14]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="../model/sample_request_file.json",
)

'["high risk"]'

<p>You can hence test the deployment by running the testDeployment.py script located at "../Consume/"<p>